# 🔁 Week 7-8 · Notebook 04 · Advanced LCEL: Building Resilient Workflows

**Module:** LangChain, Agents, & Advanced RAG  
**Project:** Manufacturing Copilot - Engineering for Failure

---

### Beyond Linear Chains: Orchestrating Complex Workflows

So far, we've built simple, linear chains: `prompt | model | parser`. But real-world applications are rarely so straightforward. They need to handle multiple steps, execute tasks in parallel, and, most importantly, be resilient to failure. What happens if a database connection drops? Or an API call times out?

This is where the full power of the **LangChain Expression Language (LCEL)** shines. LCEL is more than just a way to pipe components together; it's a complete language for orchestrating complex, non-linear workflows.

In this notebook, we will explore advanced LCEL concepts to build robust and resilient AI systems. We will learn how to:
-   Execute different parts of our chain in parallel.
-   Create fallback mechanisms to handle errors gracefully.
-   Implement a "Circuit Breaker" pattern to prevent cascading failures.

These techniques are essential for moving from simple prototypes to production-grade applications that can withstand the unpredictability of the real world.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to build sophisticated, production-ready workflows with LCEL. You will learn to:

1.  **Orchestrate Non-Linear Steps with `RunnableParallel`:** Construct chains where multiple operations (like retrieval and data formatting) can run in parallel.
2.  **Implement Resilient Fallbacks:** Use the `.with_fallbacks()` method to create chains that can gracefully handle errors by trying alternative paths.
3.  **Create Custom Logic with `RunnableLambda`:** Wrap any Python function into a reusable LCEL component.
4.  **Design a "Circuit Breaker" for Failure Prevention:** Implement a stateful pattern to temporarily halt operations after repeated failures, preventing system overloads.

### Scenario: Building a Resilient Financial Q&A System

Imagine you are building a financial services chatbot. Your customers expect fast, accurate answers to their questions. However, the primary Large Language Model (LLM) you use might occasionally fail due to high traffic, API rate limits, or other transient issues.

To ensure a seamless user experience, you need to design a system that can handle these failures gracefully. Your goal is to build a workflow that:
1.  **Tries the primary, high-performance model first.**
2.  **If the primary model fails, it automatically switches to a reliable secondary model.**
3.  **If both models fail repeatedly, it triggers a "circuit breaker"** to stop sending requests for a short period, preventing further errors and allowing the system to recover.

This notebook will guide you through implementing this resilient workflow using advanced LCEL features.

## 1. Environment Setup

First, let's install the necessary libraries. We'll need `langchain-openai` for the models, `python-dotenv` to manage our API keys securely, and `langchain` for the core framework.

> ⚠️ **Kernel Restart**: After running the installation cell below, you may need to restart the kernel for the changes to take effect. You can do this from the "Kernel" menu in your Jupyter environment.

In [ ]:
%pip install -qU langchain langchain-openai python-dotenv

Next, we'll load the API keys from a `.env` file. This is a best practice for keeping your credentials secure and out of your codebase.

Create a `.env` file in the same directory as this notebook with the following content:

```
OPENAI_API_KEY="your_openai_api_key"
```

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("OPENAI_API_KEY not found. Please set it in your .env file.")

## 2. Core Components: Prompt, Models, and Parser

Let's define the basic building blocks for our chain. We'll create a prompt template, define two different OpenAI models (a primary and a fallback), and an output parser.

-   **`ChatPromptTemplate`**: Structures the input for the model.
-   **`ChatOpenAI`**: The interface to the OpenAI models. We'll use `gpt-4o` as our powerful primary model and `gpt-3.5-turbo` as our reliable fallback.
-   **`StrOutputParser`**: A simple parser to convert the model's `AIMessage` output into a clean string.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. Prompt Template
prompt = ChatPromptTemplate.from_template(
    "You are a helpful financial assistant. Answer the following question: {question}"
)

# 2. Models
# Primary, high-performance model
primary_llm = ChatOpenAI(model="gpt-4o", temperature=0)
# Fallback, reliable model
fallback_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 3. Output Parser
output_parser = StrOutputParser()

## 3. Building a Resilient Chain with Fallbacks

The core of our resilient system is the ability to fall back to a secondary model if the primary one fails. LCEL makes this incredibly simple with the `.with_fallbacks()` method.

You can attach this method to any `Runnable` (like a model or a chain). It takes a list of alternative `Runnables` to try in order if the primary one raises an exception.

Here, we'll create a chain where `primary_llm` (`gpt-4o`) is the first choice. If it fails, the chain will automatically retry the request with `fallback_llm` (`gpt-3.5-turbo`).

In [ ]:
# Create a chain with a fallback model
resilient_chain = prompt | primary_llm.with_fallbacks([fallback_llm]) | output_parser

# Let's test it. This will use the primary model (gpt-4o)
question = "What is the current market capitalization of Apple Inc.?"
response = resilient_chain.invoke({"question": question})

print(f"Question: {question}")
print(f"Response: {response}")

### Simulating an Error to Test the Fallback

How can we be sure the fallback works? We can create a "faulty" model that is designed to fail.

Here, we'll use `RunnableLambda` to wrap a simple Python function that always raises an exception. `RunnableLambda` is a powerful tool that lets you turn any function or lambda into a `Runnable` component, making it easy to integrate custom logic into your LCEL chains.

We will place this faulty model as the primary LLM in our chain. When we invoke the chain, it will first try the faulty model, which will fail. Then, thanks to `.with_fallbacks()`, it will automatically switch to the next model in the list—our `primary_llm` (`gpt-4o`).

In [ ]:
from langchain_core.runnables import RunnableLambda

# A simple function that always raises an error
def faulty_model_func(input):
    raise ValueError("This model is intentionally broken!")

# Wrap the function in a RunnableLambda to create a "faulty" model
faulty_model = RunnableLambda(faulty_model_func)

# Build a chain where the first model is the faulty one.
# The chain will try faulty_model, fail, and then try primary_llm.
chain_with_faulty_primary = (
    prompt | faulty_model.with_fallbacks([primary_llm]) | output_parser
)

# Invoke the chain. We expect it to fail on the first try but succeed on the fallback.
response = chain_with_faulty_primary.invoke({"question": question})

print(f"Question: {question}")
print(f"Response from fallback: {response}")

## 4. Advanced Resilience: Implementing a Circuit Breaker

Fallbacks are great for handling occasional, transient errors. But what if a service is completely down? Continuously retrying a failing service can waste resources and add load to an already struggling system.

A **Circuit Breaker** is a design pattern that solves this problem. It works like an electrical circuit breaker:
1.  **Closed State:** Requests flow normally. If a request fails, it increments a failure counter.
2.  **Open State:** If the failure count exceeds a threshold within a certain time, the circuit "opens." All subsequent requests are immediately rejected without even trying, preventing further load on the failing service.
3.  **Half-Open State:** After a timeout period, the circuit moves to a "half-open" state. It allows a single request to pass through. If it succeeds, the circuit closes and returns to normal. If it fails, the circuit opens again.

We will implement a simple, stateful circuit breaker using a Python class. This class will wrap our LCEL chain and manage the state (Closed, Open).

### The `CircuitBreaker` Class
Our implementation will include:
-   `failure_threshold`: The number of failures before the circuit opens.
-   `recovery_timeout`: The time in seconds before the circuit moves to half-open.
-   `invoke()`: The main method that wraps the chain's `invoke` call, adding the circuit breaker logic.

In [ ]:
import time
from datetime import datetime, timedelta

class CircuitBreaker:
    def __init__(self, chain, failure_threshold=3, recovery_timeout=60):
        self.chain = chain
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.failure_count = 0
        self.state = "CLOSED"  # Can be CLOSED, OPEN, HALF_OPEN
        self.last_failure_time = None

    def invoke(self, *args, **kwargs):
        if self.state == "OPEN":
            # If the recovery timeout has passed, move to HALF_OPEN
            if (datetime.now() - self.last_failure_time) > timedelta(seconds=self.recovery_timeout):
                self.state = "HALF_OPEN"
            else:
                # If still within the timeout, reject the call immediately
                raise ConnectionError("Circuit is open. Please try again later.")

        try:
            # In CLOSED or HALF_OPEN state, try the chain
            result = self.chain.invoke(*args, **kwargs)
            # If the call was successful, reset the circuit
            self._reset()
            return result
        except Exception as e:
            # If the call fails, handle the failure
            self._handle_failure()
            # Re-raise the exception to the caller
            raise e

    def _handle_failure(self):
        self.failure_count += 1
        self.last_failure_time = datetime.now()
        
        # If in HALF_OPEN, a single failure re-opens the circuit
        if self.state == "HALF_OPEN":
            self.state = "OPEN"
            print("Circuit breaker failed in HALF_OPEN state. Re-opening circuit.")
        # If failure count exceeds threshold, open the circuit
        elif self.failure_count >= self.failure_threshold:
            self.state = "OPEN"
            print(f"Circuit breaker opened due to {self.failure_count} failures.")

    def _reset(self):
        # On success, reset the failure count and close the circuit
        if self.state != "CLOSED":
            print("Circuit breaker has been reset and is now CLOSED.")
        self.failure_count = 0
        self.state = "CLOSED"
        self.last_failure_time = None

### Testing the Circuit Breaker

Now, let's test our `CircuitBreaker` class. We'll wrap a chain that is designed to fail consistently.

1.  **Create a chain that always fails:** We'll use our `faulty_model` again, but this time without any fallbacks.
2.  **Wrap it with the `CircuitBreaker`:** We'll set a low `failure_threshold` (e.g., 2) and a short `recovery_timeout` (e.g., 5 seconds) for demonstration purposes.
3.  **Simulate calls:** We'll call the circuit breaker in a loop.
    -   The first two calls should fail, and the circuit will open.
    -   The third call should be immediately rejected by the open circuit.
    -   We'll wait for the recovery timeout to pass.
    -   The next call will be in the "half-open" state. Since our model is still faulty, it will fail, and the circuit will re-open.

In [ ]:
# A chain that is guaranteed to fail
failing_chain = prompt | faulty_model | output_parser

# Wrap the failing chain in our circuit breaker
# Using a short timeout for demonstration
breaker = CircuitBreaker(failing_chain, failure_threshold=2, recovery_timeout=5)

# --- Simulation ---

# 1. First two calls fail, tripping the breaker
for i in range(breaker.failure_threshold):
    try:
        print(f"Attempt {i+1}: Calling the chain...")
        breaker.invoke({"question": "This will fail."})
    except Exception as e:
        print(f"Attempt {i+1} failed as expected: {e.__class__.__name__}")

print(f"\nCircuit state is now: {breaker.state}")

# 2. The third call should be rejected immediately
try:
    print("\nAttempt 3: Calling the chain while circuit is open...")
    breaker.invoke({"question": "This should be blocked."})
except Exception as e:
    print(f"Attempt 3 was correctly blocked: {e}")

# 3. Wait for the recovery timeout
print(f"\nWaiting for {breaker.recovery_timeout} seconds for recovery...")
time.sleep(breaker.recovery_timeout)

# 4. The fourth call is in the HALF_OPEN state. It will fail and re-open the circuit.
try:
    print("\nAttempt 4: Calling in HALF_OPEN state...")
    breaker.invoke({"question": "This will fail again."})
except Exception as e:
    print(f"Attempt 4 failed as expected: {e.__class__.__name__}")

print(f"\nFinal circuit state: {breaker.state}")

In [ ]:
import time
import random
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# --- Step 1: Define Individual Components ---

# Simulate a retrieval function that might fail
def fetch_retrieval(context_key: str) -> str:
    """
    Simulates fetching a document from a vector store.
    Set to fail for this demonstration.
    """
    print(f"Attempting to retrieve document for: {context_key}")
    # Simulate failure
    raise ConnectionError("Retrieval failed: Vector DB is offline.")
    # In a success case, it would return:
    # return "SOP-455 states that vibration above 5mm/s requires immediate shutdown and inspection."

# A fallback function that provides a cached or default response
def fallback_cache(inputs: dict) -> str:
    """
    Provides a safe, cached response when the primary retrieval fails.
    """
    print("--- Primary retrieval failed. Using fallback cache. ---")
    issue = inputs.get("issue", "the reported issue")
    return f"Cached Guidance: For {issue}, consult the general troubleshooting manual (GTM-001) and notify the shift supervisor."

# --- Step 2: Build the Runnable Sequence using LCEL ---

# The main prompt for the LLM
prompt = ChatPromptTemplate.from_template(
    "Context: {context}\n\nIssue: {issue}\n\nAnswer succinctly with SOP citations if available."
)

# The LLM to use for generation
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Define the primary generation chain
generation_chain = prompt | llm | StrOutputParser()

# Define the retrieval chain with a fallback
# .with_fallbacks() creates a resilient chain that tries the primary runnable first
# and executes the fallback if the primary one fails.
retrieval_with_fallback = RunnableLambda(fetch_retrieval).with_fallbacks(
    fallbacks=[RunnableLambda(fallback_cache)]
)

# --- Step 3: Compose the final workflow ---

# The final workflow uses a RunnableParallel to structure the input for the generation_chain.
# It runs the retrieval_with_fallback and passes the 'issue' straight through.
workflow = (
    RunnableParallel(
        context=retrieval_with_fallback,
        issue=lambda inputs: inputs["issue"]
    )
    | generation_chain
)


# --- Step 4: Execute and Monitor ---

start = time.perf_counter()

# The input dictionary for the workflow
run_input = {
    "context_key": "vibration_alarm_press_12",
    "issue": "High vibration alarm on Press 12"
}

# Invoke the workflow
final_answer = workflow.invoke(run_input)

latency_ms = (time.perf_counter() - start) * 1000

print("\n--- Final Answer ---")
print(final_answer)
print(f'\nLatency: {latency_ms:.1f} ms')

### ⚠️ Circuit Breaker Pattern
Track failure counts; if > 3 within 10 minutes, disable automated responses and notify SMEs.

In [ ]:
from datetime import datetime, timedelta

class CircuitBreaker:
    """
    A simple circuit breaker implementation to prevent repeated failures.
    """
    def __init__(self, failure_threshold: int, recovery_timeout_seconds: int):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = timedelta(seconds=recovery_timeout_seconds)
        self.failures = []
        self.is_open = False
        self.last_opened_time = None

    def record_failure(self):
        """Records a failure and opens the circuit if the threshold is met."""
        now = datetime.now()
        self.failures.append(now)
        
        # Remove old failures
        self.failures = [t for t in self.failures if now - t < self.recovery_timeout]
        
        if len(self.failures) >= self.failure_threshold:
            self.is_open = True
            self.last_opened_time = now
            print(f"CIRCUIT BREAKER OPENED at {now}. Further calls will be blocked.")

    def can_execute(self) -> bool:
        """Checks if the circuit is closed or if the recovery timeout has passed."""
        if not self.is_open:
            return True
        
        if datetime.now() - self.last_opened_time > self.recovery_timeout:
            self.reset()
            print("CIRCUIT BREAKER RESET. Calls are now permitted.")
            return True
            
        print("Circuit breaker is open. Call is blocked.")
        return False

    def reset(self):
        """Resets the circuit breaker to a closed state."""
        self.is_open = False
        self.failures = []
        self.last_opened_time = None

# --- Example Usage ---
breaker = CircuitBreaker(failure_threshold=3, recovery_timeout_seconds=60)

for i in range(5):
    if breaker.can_execute():
        print(f"Attempt {i+1}: Executing the operation...")
        # Simulate a failure
        breaker.record_failure()
        time.sleep(1) # Simulate time between calls
    else:
        print(f"Attempt {i+1}: Operation blocked by circuit breaker.")
        # In a real app, you would wait or redirect here
        time.sleep(10)

# Check if it resets after timeout (manual check for demo)
print("\n--- Waiting for recovery timeout ---")
# time.sleep(61) 
# if breaker.can_execute():
#     print("System recovered and is operational again.")

## 🧪 Lab Assignment
1. Add a parallel branch that translates outputs into Spanish using `RunnableParallel`.
2. Extend the fallback to fetch last known guidance from Redis when retrieval fails.
3. Log per-node latency and completion status to Prometheus.
4. Demo circuit breaker resetting after manual override by SME.

## ✅ Checklist
- [ ] Runnable sequence deployed
- [ ] Failure handling strategy documented
- [ ] Latency metrics instrumented
- [ ] Lab deliverables verified with ops team

## 📚 References
- LangChain Runnables Guide
- Site Reliability Engineering: Circuit Breakers
- Week 09-10 Monitoring Notebook